In [30]:
import pandas as pd
import numpy as np

# Contribución empírica del crecimiento económico: Estimaciones OLS, CLS y EMD

1. Carga la base de datos en un DataFrame.

In [31]:
url=r'C:\Users\Camil\Documents\GitHub\ECOP2037_CE\hmw4\MRW1992.dta'
df=pd.read_stata(url)

2. Calcula las matrices X (con columna de 1s) y y a partir del DataFrame.

In [43]:
df = df[df['N'] != 0]

log_change=np.log(df['Y85'])-np.log(df['Y60'])

logGDP60=np.log(df['Y60'])

logI_GDP=np.log(df['invest']/100)

log_school=np.log(df['school']/100)

log_growth=np.log(df['pop_growth']/100 +0.05)

In [46]:
y=np.array(log_change)
y=y.reshape(-1, 1)

x=np.column_stack([np.ones(len(df)), logGDP60, logI_GDP, log_growth,log_school])

3. Implementa el estimador OLS y sus su matriz de varianza y covarianza HCO.

In [51]:
beta_ols= (np.linalg.inv(x.T @ x)) @ (x.T @ y)
residuos = y - x @ beta_ols

n, k = x.shape
m = np.zeros((k, k))
for i in range(n):
    xi = x[i, :].reshape(-1, 1)
    ui2 = residuos[i]**2         
    m += ui2 * (xi @ xi.T)          

var_covOLS= (np.linalg.inv(x.T @ x)) @ m @ np.linalg.inv(x.T @ x)


4. Implementa el estimador CLS y su matriz de varianza y covarianza.

In [ ]:
R = np.array([[0, 0, 1, 1, 1]])
r = np.array([[0]])

beta_cls= beta_ols- [(np.linalg.inv(x.T @x)) @ R.T @ np.linalg.inv(R @ np.linalg.inv(x.T @x) @ R.T) @ ((R@beta_ols) -r )]

residuosCLS = (y-x@beta_cls).reshape(-1, 1)
n, k = x.shape
q = R.shape[0]
sigma2_hat = (residuosCLS.T @ residuosCLS) / (n - k + q)

m = np.linalg.inv(R @ (np.linalg.inv(x.T @x)) @ R.T)
correction = (np.linalg.inv(x.T @x)) @ R.T @ m @ R @ (np.linalg.inv(x.T @x))

var_covCLS = sigma2_hat.item() * ((np.linalg.inv(x.T @x)) - correction)

5. Implementa el estimador EMD. Pondera por una matriz de varianza estimada eficiente bV ,
siguiendo el mismo principio de CLS. Estima también la matriz de varianza y covarianza.